In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 200)

In [3]:
def create_column_dict(path):
    """
    Reads in the two rows of column names from the data set. 
    The first row is a descriptive column name while the second row is simpler unique id.
    Creates a dictionary of the two sets of column names.
    Ignores the first 6 column names which are descriptive in both rows.
    Simplifies the formatting and types of characters within the descriptive column names.
    Returns the dictionary.
    """
    # read in the first two rows
    columns_df = pd.read_csv(path, nrows=1)
    # create the dictionary
    columns_dict = columns_df.iloc[0, 7:].to_dict()
    # swap the keys and values while also removing capitalizations
    columns_dict = dict([(value, key.lower())
                         for key, value in columns_dict.items()])
    # loop through the dictionary to make various string replacements
    str_replace_list = [(' - ', '_'), (' ', '_'), ('-', '_'), ('/', '_'), ('=', '_eqls_'),
                        ('(', ''), (')', ''), ('%', 'pct'), ('+', 'plus'), ('.', '')]
    for key, value in columns_dict.items():
        for replacement in str_replace_list:
            value = value.replace(replacement[0], replacement[1])
        columns_dict.update({key: value})

    return columns_dict

In [4]:
data_path = 'data/analytic_data2019.csv'
columns_dict = create_column_dict(data_path)

In [5]:
health_df = pd.read_csv('data/analytic_data2019.csv', skiprows=1)

In [6]:
# dropping columns without any values
health_df.dropna(axis=1, how='all', inplace=True)

In [7]:
def drop_ci_num_denom(df):
    """
    Drops columns that contain Confidence Intervals (ci) 
    as well as numerators (num) and denominators (denom) used to calculate other values.
    Returns a copy of the DataFrame (df)
    """
    drop_cols = []
    for col in df.columns:
        if (col.find('cilow')
            + col.find('cihigh')
            + col.find('numerator')
            + col.find('denominator')) > -4:
            drop_cols.append(col)
    return df.drop(axis=1, columns=drop_cols)

In [8]:
health_df = drop_ci_num_denom(health_df)

In [9]:
# renaming the columns to be more interpretable
health_df.rename(columns=columns_dict, inplace=True)

In [10]:
# dropping rows that are not county ranked as they have less data and no premature death info
health_df = health_df.loc[health_df.county_ranked != 0].copy()
health_df.reset_index(inplace=True, drop=True)

In [11]:
# Creating another dataframe with counts of missing values for exploration
def column_na_count_df(df):
    """
    Generates a dataframe with one column listing every column in the DataFrame parameter (df),
    and another column with the count of na's in the column.
    """
    na_dict = {}
    for col in df.columns:
        na_count = df[col].isna().sum()
        na_dict.update({col: na_count})
    na_count_df = pd.DataFrame.from_dict(data=na_dict, 
                                         orient='index',
                                         columns=['na_count'])
    na_count_df.reset_index(inplace=True)
    na_count_df.rename(columns={'index': 'column'}, inplace=True)
    return na_count_df

In [12]:
na_count_df = column_na_count_df(health_df)
na_count_df

column  na_count
0                                            statecode         0
1                                           countycode         0
2                                             fipscode         0
3                                                state         0
4                                               county         0
5                                                 year         0
6                                        county_ranked        52
7                            premature_death_raw_value         0
8                                premature_death_black      1779
9                             premature_death_hispanic      2294
10                               premature_death_white      1551
11                       poor_or_fair_health_raw_value         1
12                 poor_physical_health_days_raw_value         1
13                   poor_mental_health_days_raw_value         1
14                           low_birthweight_raw_value        53
15                               low_birthweight_black      1743
16                            low_birthweight_hispanic      1728
17                               low_birthweight_white      1316
18                             adult_smoking_raw_value         1
19                             adult_obesity_raw_value         0
20                    food_environment_index_raw_value        19
21                       physical_inactivity_raw_value         0
22          access_to_exercise_opportunities_raw_value         5
23                        excessive_drinking_raw_value         1
24           alcohol_impaired_driving_deaths_raw_value        22
25           sexually_transmitted_infections_raw_value       105
26                               teen_births_raw_value        86
27                                   teen_births_black      1744
28                                teen_births_hispanic      1557
29                                   teen_births_white      1206
30                                 uninsured_raw_value         0
31                   primary_care_physicians_raw_value       134
32      ratio_of_population_to_primary_care_physicians       134
33                                  dentists_raw_value        85
34                     ratio_of_population_to_dentists        85
35                   mental_health_providers_raw_value       230
36      ratio_of_population_to_mental_health_providers       230
37                preventable_hospital_stays_raw_value        18
38                    preventable_hospital_stays_black      1651
39                 preventable_hospital_stays_hispanic      2115
40                    preventable_hospital_stays_white      1357
41                     mammography_screening_raw_value        12
42                         mammography_screening_black      1587
43                      mammography_screening_hispanic      1751
44                         mammography_screening_white      1166
45                          flu_vaccinations_raw_value        11
46                              flu_vaccinations_black      1282
47                           flu_vaccinations_hispanic      1301
48                              flu_vaccinations_white       840
49                    high_school_graduation_raw_value        78
50                              some_college_raw_value         0
51                              unemployment_raw_value         0
52                       children_in_poverty_raw_value         0
53                           children_in_poverty_black      1127
54                        children_in_poverty_hispanic       480
55                           children_in_poverty_white       319
56                         income_inequality_raw_value         0
57      children_in_single_parent_households_raw_value         0
58                       social_associations_raw_value         0
59                             violent_crime_raw_value       173
60                             injury_deaths_raw_value        47
61          air_po

In [13]:
na_count_df[na_count_df.na_count >= 3060]

column  na_count
125                     communicable_disease_raw_value      3060
126   self_inflicted_injury_hospitalizations_raw_value      3063
127                         cancer_incidence_raw_value      3060
128  coronary_heart_disease_hospitalizations_raw_value      3062
129  cerebrovascular_disease_hospitalizations_raw_v...      3062
130                 smoking_during_pregnancy_raw_value      3060
131                             drug_arrests_raw_value      3061
132                   opioid_hospital_visits_raw_value      3061
133         alcohol_related_hospitalizations_raw_value      3070
134       motor_vehicle_crash_occupancy_rate_raw_value      3060
135  on_road_motor_vehicle_crash_related_er_visits_...      3060
136  off_road_motor_vehicle_crash_related_er_visits...      3061
137                  childhood_immunizations_raw_value      3060
138                      reading_proficiency_raw_value      3060
139                           w_2_enrollment_raw_value      3060
140                                  poverty_raw_value      3060
141                older_adults_living_alone_raw_value      3060
142                              hate_crimes_raw_value      3120
143                              child_abuse_raw_value      3060
144                  injury_hospitalizations_raw_value      3060
145                   fall_fatalities_65plus_raw_value      3068
146                     year_structure_built_raw_value      3060
147                     male_population_0_17_raw_value      3060
148                    male_population_18_44_raw_value      3060
149                    male_population_45_64_raw_value      3060
150                   male_population_65plus_raw_value      3060
151                    total_male_population_raw_value      3060
152                   female_population_0_17_raw_value      3060
153                  female_population_18_44_raw_value      3060
154                  female_population_45_64_raw_value      3060
155                 female_population_65plus_raw_value      3060
156                  total_female_population_raw_value      3060
157                        population_growth_raw_value      3060

In [14]:
# Decided to drop race-based data as it is missing from many records
# Also decided to drop when na is greater than or equal to 3060 as this represents a pattern among the last 33 columns,
# which represent all the columns satisfying this condition

In [15]:
def drop_race_based_data(df):
    """
    Drops columns that contain a given metric based on race.
    Returns a copy of the DataFrame (df)
    """
    drop_cols = []
    for col in df.columns:
        if (col.find('cilow')
            + col.find('black')
            + col.find('hispanic')
            + col.find('white')) > -3:
            drop_cols.append(col)
    return df.drop(axis=1, columns=drop_cols)

In [16]:
health_df = drop_race_based_data(health_df)

In [17]:
na_threshold = health_df.shape[0] - 3059
health_df.dropna(axis=1, thresh=na_threshold, inplace=True)

In [18]:
health_df

statecode  countycode  fipscode state             county  year  \
0             0           0         0    US      United States  2019   
1             1           0      1000    AL            Alabama  2019   
2             1           1      1001    AL     Autauga County  2019   
3             1           3      1003    AL     Baldwin County  2019   
4             1           5      1005    AL     Barbour County  2019   
...         ...         ...       ...   ...                ...   ...   
3128         56          37     56037    WY  Sweetwater County  2019   
3129         56          39     56039    WY       Teton County  2019   
3130         56          41     56041    WY       Uinta County  2019   
3131         56          43     56043    WY    Washakie County  2019   
3132         56          45     56045    WY      Weston County  2019   

      county_ranked  premature_death_raw_value  poor_or_fair_health_raw_value  \
0               NaN                6900.630354                            NaN   
1               NaN                9917.232898                       0.214024   
2               1.0                8824.057123                       0.184111   
3               1.0                7224.632160                       0.180605   
4               1.0                9586.165037                       0.257734   
...             ...                        ...                            ...   
3128            1.0                7497.439952                       0.153772   
3129            1.0                3786.128226                       0.121817   
3130            1.0                7790.302043                       0.158858   
3131            1.0                5504.650970                       0.161261   
3132            1.0                4991.168273                       0.132375   

      poor_physical_health_days_raw_value  ...  \
0                                     NaN  ...   
1                                4.400458  ...   
2                                4.200578  ...   
3                                4.098748  ...   
4                                5.067438  ...   
...                                   ...  ...   
3128                             3.536556  ...   
3129                             3.166316  ...   
3130                             3.677538  ...   
3131                             3.601687  ...   
3132                             3.448064  ...   

      severe_housing_cost_burden_raw_value  population_raw_value  \
0                                 0.151242             325719178   
1                                 0.129176               4874747   
2                                 0.126452                 55504   
3                                 0.127959                212628   
4                                 0.135868                 25270   
...                                    ...                   ...   
3128                              0.092849                 43534   
3129                              0.105918                 23265   
3130                              0.084049                 20495   
3131                              0.089367                  8064   
3132                              0.122541                  6927   

      pct_below_18_years_of_age_raw_value  pct_65_and_older_raw_value  \
0                                0.226132                    0.156143   
1                                0.224724                    0.164885   
2                                0.239370                    0.151196   
3                                0.218485                    0.199470   
4                                0.207638                    0.188247   
...                                   ...                         ...   
3128                             0.265172                    0.113750   
3129                             0.187965                    0.143950   
3130                             0.292120                    0.131886   
3131                

In [19]:
county_health_df = health_df[health_df.county_ranked == 1].copy()

In [61]:
county_na_count_df = column_na_count_df(county_health_df)

In [62]:
county_na_count_df

column  na_count
0                                           statecode         0
1                                          countycode         0
2                                            fipscode         0
3                                               state         0
4                                              county         0
5                                                year         0
6                                       county_ranked         0
7                           premature_death_raw_value         0
8                       poor_or_fair_health_raw_value         0
9                 poor_physical_health_days_raw_value         0
10                  poor_mental_health_days_raw_value         0
11                          low_birthweight_raw_value        53
12                            adult_smoking_raw_value         0
13                            adult_obesity_raw_value         0
14                   food_environment_index_raw_value        19
15                      physical_inactivity_raw_value         0
16         access_to_exercise_opportunities_raw_value         5
17                       excessive_drinking_raw_value         0
18          alcohol_impaired_driving_deaths_raw_value        22
19          sexually_transmitted_infections_raw_value       105
20                              teen_births_raw_value        86
21                                uninsured_raw_value         0
22                  primary_care_physicians_raw_value       134
23     ratio_of_population_to_primary_care_physicians       134
24                                 dentists_raw_value        85
25                    ratio_of_population_to_dentists        85
26                  mental_health_providers_raw_value       230
27     ratio_of_population_to_mental_health_providers       230
28               preventable_hospital_stays_raw_value        17
29                    mammography_screening_raw_value        11
30                         flu_vaccinations_raw_value        10
31                   high_school_graduation_raw_value        77
32                             some_college_raw_value         0
33                             unemployment_raw_value         0
34                      children_in_poverty_raw_value         0
35                        income_inequality_raw_value         0
36     children_in_single_parent_households_raw_value         0
37                      social_associations_raw_value         0
38                            violent_crime_raw_value       173
39                            injury_deaths_raw_value        47
40         air_pollution_particulate_matter_raw_value        29
41                drinking_water_violations_raw_value        42
42                  severe_housing_problems_raw_value         0
43   percentage_of_households_with_high_housing_costs         0
44         percentage_of_households_with_overcrowding         0
45  percentage_of_households_with_lack_of_kitchen_...         0
46                    driving_alone_to_work_raw_value         0
47               long_commute_driving_alone_raw_value         0
48                          life_expectancy_raw_value        20
49         premature_age_adjusted_mortality_raw_value         0
50                          child_mortality_raw_value      1161
51                         infant_mortality_raw_value      1821
52               frequent_physical_distress_raw_value         0
53                 frequent_mental_distress_raw_value         0
54                      diabetes_prevalence_raw_value         0
55                           hiv_prevalence_raw_value       660
56                          food_insecurity_raw_value         0
57          limited_access_to_healthy_foods_raw_value        19
58                     drug_overdose_deaths_raw_value      1361
59               motor_vehicle_crash_deaths_raw_value       382
60                       insufficient_sleep_raw_value         0
61                         uninsured_adults_raw_value         0
62             

In [63]:
print(county_na_count_df[county_na_count_df.na_count == 0].shape[0])
county_na_count_df[county_na_count_df.na_count == 0]

48


column  na_count
0                                           statecode         0
1                                          countycode         0
2                                            fipscode         0
3                                               state         0
4                                              county         0
5                                                year         0
6                                       county_ranked         0
7                           premature_death_raw_value         0
8                       poor_or_fair_health_raw_value         0
9                 poor_physical_health_days_raw_value         0
10                  poor_mental_health_days_raw_value         0
12                            adult_smoking_raw_value         0
13                            adult_obesity_raw_value         0
15                      physical_inactivity_raw_value         0
17                       excessive_drinking_raw_value         0
21                                uninsured_raw_value         0
32                             some_college_raw_value         0
33                             unemployment_raw_value         0
34                      children_in_poverty_raw_value         0
35                        income_inequality_raw_value         0
36     children_in_single_parent_households_raw_value         0
37                      social_associations_raw_value         0
42                  severe_housing_problems_raw_value         0
43   percentage_of_households_with_high_housing_costs         0
44         percentage_of_households_with_overcrowding         0
45  percentage_of_households_with_lack_of_kitchen_...         0
46                    driving_alone_to_work_raw_value         0
47               long_commute_driving_alone_raw_value         0
49         premature_age_adjusted_mortality_raw_value         0
52               frequent_physical_distress_raw_value         0
53                 frequent_mental_distress_raw_value         0
54                      diabetes_prevalence_raw_value         0
56                          food_insecurity_raw_value         0
60                       insufficient_sleep_raw_value         0
61                         uninsured_adults_raw_value         0
62                       uninsured_children_raw_value         0
66                  median_household_income_raw_value         0
70                            homeownership_raw_value         0
71               severe_housing_cost_burden_raw_value         0
72                               population_raw_value         0
73                pct_below_18_years_of_age_raw_value         0
74                         pct_65_and_older_raw_value         0
75   pct_american_indian_and_alaskan_native_raw_value         0
76                                pct_asian_raw_value         0
77  pct_native_hawaiian_other_pacific_islander_raw...         0
78            pct_not_proficient_in_english_raw_value         0
79                              pct_females_raw_value         0
80                                pct_rural_raw_value         0

In [27]:
# All places missing pct_rural are in alaska and south dakota, presumably they are entirely rural
county_health_df[county_health_df.pct_rural_raw_value.isna()]

Empty DataFrame
Columns: [statecode, countycode, fipscode, state, county, year, county_ranked, premature_death_raw_value, poor_or_fair_health_raw_value, poor_physical_health_days_raw_value, poor_mental_health_days_raw_value, low_birthweight_raw_value, adult_smoking_raw_value, adult_obesity_raw_value, food_environment_index_raw_value, physical_inactivity_raw_value, access_to_exercise_opportunities_raw_value, excessive_drinking_raw_value, alcohol_impaired_driving_deaths_raw_value, sexually_transmitted_infections_raw_value, teen_births_raw_value, uninsured_raw_value, primary_care_physicians_raw_value, ratio_of_population_to_primary_care_physicians, dentists_raw_value, ratio_of_population_to_dentists, mental_health_providers_raw_value, ratio_of_population_to_mental_health_providers, preventable_hospital_stays_raw_value, mammography_screening_raw_value, flu_vaccinations_raw_value, high_school_graduation_raw_value, some_college_raw_value, unemployment_raw_value, children_in_poverty_raw_value, income_inequality_raw_value, children_in_single_parent_households_raw_value, social_associations_raw_value, violent_crime_raw_value, injury_deaths_raw_value, air_pollution_particulate_matter_raw_value, drinking_water_violations_raw_value, severe_housing_problems_raw_value, percentage_of_households_with_high_housing_costs, percentage_of_households_with_overcrowding, percentage_of_households_with_lack_of_kitchen_or_plumbing_facilities, driving_alone_to_work_raw_value, long_commute_driving_alone_raw_value, life_expectancy_raw_value, premature_age_adjusted_mortality_raw_value, child_mortality_raw_value, infant_mortality_raw_value, frequent_physical_distress_raw_value, frequent_mental_distress_raw_value, diabetes_prevalence_raw_value, hiv_prevalence_raw_value, food_insecurity_raw_value, limited_access_to_healthy_foods_raw_value, drug_overdose_deaths_raw_value, motor_vehicle_crash_deaths_raw_value, insufficient_sleep_raw_value, uninsured_adults_raw_value, uninsured_children_raw_value, other_primary_care_providers_raw_value, ratio_of_population_to_primary_care_providers_other_than_physicians, disconnected_youth_raw_value, median_household_income_raw_value, children_eligible_for_free_or_reduced_price_lunch_raw_value, homicides_raw_value, firearm_fatalities_raw_value, homeownership_raw_value, severe_housing_cost_burden_raw_value, population_raw_value, pct_below_18_years_of_age_raw_value, pct_65_and_older_raw_value, pct_american_indian_and_alaskan_native_raw_value, pct_asian_raw_value, pct_native_hawaiian_other_pacific_islander_raw_value, pct_not_proficient_in_english_raw_value, pct_females_raw_value, pct_rural_raw_value]
Index: []

[0 rows x 81 columns]

In [26]:
county_health_df.pct_rural_raw_value.fillna(value=1, inplace=True)

In [24]:
# values are all numbers, which is good
county_health_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3081 entries, 2 to 3132
Data columns (total 81 columns):
statecode                                                               3081 non-null int64
countycode                                                              3081 non-null int64
fipscode                                                                3081 non-null int64
state                                                                   3081 non-null object
county                                                                  3081 non-null object
year                                                                    3081 non-null int64
county_ranked                                                           3081 non-null float64
premature_death_raw_value                                               3081 non-null float64
poor_or_fair_health_raw_value                                           3081 non-null float64
poor_physical_health_days_raw_value                      

In [22]:
# the data set currently includes sub-totals at the state and national level
# county_ranked is blank for the sub-totals

In [73]:
county_health_df.population_raw_value[county_health_df.low_birthweight_raw_value.isna()].max()

5671

In [74]:
for pop in np.linspace(0,10000,11):
    temp = column_na_count_df(county_health_df[county_health_df.population_raw_value > pop])
    print("Population limit: ",pop)
    print("Number of rows: ",county_health_df[county_health_df.population_raw_value > pop].shape[0])
    temp_cols = set(temp.column[temp.na_count == 0])
    orig_cols = set(county_na_count_df.column[county_na_count_df.na_count == 0])
    added_cols = list(temp_cols - orig_cols)
    print("New columns with no NaNs", added_cols)

Population limit:  0.0
Number of rows:  3081
New columns with no NaNs []
Population limit:  1000.0
Number of rows:  3079
New columns with no NaNs []
Population limit:  2000.0
Number of rows:  3043
New columns with no NaNs ['life_expectancy_raw_value']
Population limit:  3000.0
Number of rows:  2967
New columns with no NaNs ['life_expectancy_raw_value']
Population limit:  4000.0
Number of rows:  2902
New columns with no NaNs ['life_expectancy_raw_value']
Population limit:  5000.0
Number of rows:  2832
New columns with no NaNs ['life_expectancy_raw_value']
Population limit:  6000.0
Number of rows:  2748
New columns with no NaNs ['life_expectancy_raw_value', 'low_birthweight_raw_value']
Population limit:  7000.0
Number of rows:  2671
New columns with no NaNs ['life_expectancy_raw_value', 'low_birthweight_raw_value']
Population limit:  8000.0
Number of rows:  2593
New columns with no NaNs ['life_expectancy_raw_value', 'low_birthweight_raw_value']
Population limit:  9000.0
Number of rows:  

In [76]:
county_health_df[county_health_df.population_raw_value > 6000].shape[0]

2748

In [79]:
big_county_na_count_df = column_na_count_df(county_health_df[county_health_df.population_raw_value > 6000])

In [80]:
big_county_na_count_df

column  na_count
0                                           statecode         0
1                                          countycode         0
2                                            fipscode         0
3                                               state         0
4                                              county         0
5                                                year         0
6                                       county_ranked         0
7                           premature_death_raw_value         0
8                       poor_or_fair_health_raw_value         0
9                 poor_physical_health_days_raw_value         0
10                  poor_mental_health_days_raw_value         0
11                          low_birthweight_raw_value         0
12                            adult_smoking_raw_value         0
13                            adult_obesity_raw_value         0
14                   food_environment_index_raw_value        18
15                      physical_inactivity_raw_value         0
16         access_to_exercise_opportunities_raw_value         2
17                       excessive_drinking_raw_value         0
18          alcohol_impaired_driving_deaths_raw_value         6
19          sexually_transmitted_infections_raw_value         8
20                              teen_births_raw_value         3
21                                uninsured_raw_value         0
22                  primary_care_physicians_raw_value        58
23     ratio_of_population_to_primary_care_physicians        58
24                                 dentists_raw_value        50
25                    ratio_of_population_to_dentists        50
26                  mental_health_providers_raw_value        77
27     ratio_of_population_to_mental_health_providers        77
28               preventable_hospital_stays_raw_value         6
29                    mammography_screening_raw_value         6
30                         flu_vaccinations_raw_value         5
31                   high_school_graduation_raw_value        24
32                             some_college_raw_value         0
33                             unemployment_raw_value         0
34                      children_in_poverty_raw_value         0
35                        income_inequality_raw_value         0
36     children_in_single_parent_households_raw_value         0
37                      social_associations_raw_value         0
38                            violent_crime_raw_value       136
39                            injury_deaths_raw_value         1
40         air_pollution_particulate_matter_raw_value        20
41                drinking_water_violations_raw_value        39
42                  severe_housing_problems_raw_value         0
43   percentage_of_households_with_high_housing_costs         0
44         percentage_of_households_with_overcrowding         0
45  percentage_of_households_with_lack_of_kitchen_...         0
46                    driving_alone_to_work_raw_value         0
47               long_commute_driving_alone_raw_value         0
48                          life_expectancy_raw_value         0
49         premature_age_adjusted_mortality_raw_value         0
50                          child_mortality_raw_value       831
51                         infant_mortality_raw_value      1488
52               frequent_physical_distress_raw_value         0
53                 frequent_mental_distress_raw_value         0
54                      diabetes_prevalence_raw_value         0
55                           hiv_prevalence_raw_value       363
56                          food_insecurity_raw_value         0
57          limited_access_to_healthy_foods_raw_value        18
58                     drug_overdose_deaths_raw_value      1028
59               motor_vehicle_crash_deaths_raw_value       132
60                       insufficient_sleep_raw_value         0
61                         uninsured_adults_raw_value         0
62             

In [81]:
print(big_county_na_count_df[big_county_na_count_df.na_count == 0].shape[0])
big_county_na_count_df[big_county_na_count_df.na_count == 0]

50


column  na_count
0                                           statecode         0
1                                          countycode         0
2                                            fipscode         0
3                                               state         0
4                                              county         0
5                                                year         0
6                                       county_ranked         0
7                           premature_death_raw_value         0
8                       poor_or_fair_health_raw_value         0
9                 poor_physical_health_days_raw_value         0
10                  poor_mental_health_days_raw_value         0
11                          low_birthweight_raw_value         0
12                            adult_smoking_raw_value         0
13                            adult_obesity_raw_value         0
15                      physical_inactivity_raw_value         0
17                       excessive_drinking_raw_value         0
21                                uninsured_raw_value         0
32                             some_college_raw_value         0
33                             unemployment_raw_value         0
34                      children_in_poverty_raw_value         0
35                        income_inequality_raw_value         0
36     children_in_single_parent_households_raw_value         0
37                      social_associations_raw_value         0
42                  severe_housing_problems_raw_value         0
43   percentage_of_households_with_high_housing_costs         0
44         percentage_of_households_with_overcrowding         0
45  percentage_of_households_with_lack_of_kitchen_...         0
46                    driving_alone_to_work_raw_value         0
47               long_commute_driving_alone_raw_value         0
48                          life_expectancy_raw_value         0
49         premature_age_adjusted_mortality_raw_value         0
52               frequent_physical_distress_raw_value         0
53                 frequent_mental_distress_raw_value         0
54                      diabetes_prevalence_raw_value         0
56                          food_insecurity_raw_value         0
60                       insufficient_sleep_raw_value         0
61                         uninsured_adults_raw_value         0
62                       uninsured_children_raw_value         0
66                  median_household_income_raw_value         0
70                            homeownership_raw_value         0
71               severe_housing_cost_burden_raw_value         0
72                               population_raw_value         0
73                pct_below_18_years_of_age_raw_value         0
74                         pct_65_and_older_raw_value         0
75   pct_american_indian_and_alaskan_native_raw_value         0
76                                pct_asian_raw_value         0
77  pct_native_hawaiian_other_pacific_islander_raw...         0
78            pct_not_proficient_in_english_raw_value         0
79                              pct_females_raw_value         0
80                                pct_rural_raw_value         0

In [94]:
big_county_health_df = county_health_df[county_health_df.population_raw_value > 6000].copy()

In [95]:
# We independently identified Bedford County's injury_deaths_raw_value from the same data source that the 
# dataset used and it was inline with expectations (Bedford value = 80.8 and state average is 77.2)
bedford_fill = 80.79616282
bedford_index = big_county_health_df[big_county_health_df.fipscode == 51019].index
big_county_health_df.at[bedford_index, 'injury_deaths_raw_value'] = bedford_fill

In [96]:
cleaned_health_df = big_county_health_df.dropna(axis=1, how='any')

In [99]:
cleaned_health_df.head()

statecode  countycode  fipscode state          county  year  county_ranked  \
2          1           1      1001    AL  Autauga County  2019            1.0   
3          1           3      1003    AL  Baldwin County  2019            1.0   
4          1           5      1005    AL  Barbour County  2019            1.0   
5          1           7      1007    AL     Bibb County  2019            1.0   
6          1           9      1009    AL   Blount County  2019            1.0   

   premature_death_raw_value  poor_or_fair_health_raw_value  \
2                8824.057123                       0.184111   
3                7224.632160                       0.180605   
4                9586.165037                       0.257734   
5               11783.543675                       0.199969   
6               10908.101822                       0.210953   

   poor_physical_health_days_raw_value  ...  \
2                             4.200578  ...   
3                             4.098748  ...   
4                             5.067438  ...   
5                             4.363377  ...   
6                             4.512753  ...   

   severe_housing_cost_burden_raw_value  population_raw_value  \
2                              0.126452                 55504   
3                              0.127959                212628   
4                              0.135868                 25270   
5                              0.087339                 22668   
6                              0.081945                 58013   

   pct_below_18_years_of_age_raw_value  pct_65_and_older_raw_value  \
2                             0.239370                    0.151196   
3                             0.218485                    0.199470   
4                             0.207638                    0.188247   
5                             0.206061                    0.160226   
6                             0.233499                    0.178426   

   pct_american_indian_and_alaskan_native_raw_value  pct_asian_raw_value  \
2                                          0.004756             0.012792   
3                                          0.007760             0.011565   
4                                          0.006529             0.004630   
5                                          0.004279             0.002206   
6                                          0.006326             0.003017   

   pct_native_hawaiian_other_pacific_islander_raw_value  \
2                                           0.001045      
3                                           0.000687      
4                                           0.001860      
5                                           0.001147      
6                                           0.001172      

   pct_not_proficient_in_english_raw_value  pct_females_raw_value  \
2                                 0.008285               0.513422   
3                                 0.004545               0.514528   
4                                 0.011993               0.472299   
5                                 0.003943               0.464531   
6                                 0.018700               0.506886   

   pct_rural_raw_value  
2             0.420022  
3             0.422791  
4             0.677896  
5             0.683526  
6             0.899515  

[5 rows x 51 columns]

In [100]:
county_health_df.to_csv('cleaned_county_health_data.csv', index=False)